In [2]:
# Install required packages
from IPython.display import clear_output
!pip install tensorflow tensorflow_hub opencv-python-headless matplotlib kagglehub
clear_output()

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt


In [5]:
# Paths
dataset_path = "/teamspace/studios/this_studio/.cache/kagglehub/datasets/aryashah2k/large-scale-multicamera-detection-dataset/versions/1/Wildtrack"
image_dir = os.path.join(dataset_path, "Image_subsets")
annotation_dir = os.path.join(dataset_path, "annotations_positions")
calibration_dir = os.path.join(dataset_path, "calibrations")

# Check paths
print("Image directory exists:", os.path.exists(image_dir))
print("Annotation directory exists:", os.path.exists(annotation_dir))
print("Calibration directory exists:", os.path.exists(calibration_dir))


Image directory exists: True
Annotation directory exists: True
Calibration directory exists: True


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 46901085369920298
xla_global_id: -1
]


W0000 00:00:1730794004.718793    2582 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
# Load the Faster R-CNN model from TensorFlow Hub
detector = hub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
W0000 00:00:1730793577.552737   33934 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [24]:
# Helper function to process an image and run detection
def detect_objects_in_image(image):
    # Convert the image to tensor and expand dimensions
    image_tensor = tf.convert_to_tensor(image, dtype=tf.uint8)
    image_tensor = tf.image.resize(image_tensor, (640, 640))
    image_tensor = tf.expand_dims(image_tensor, axis=0)
    
    # Run object detection
    detections = detector.signatures['default'](image_tensor)
    
    return detections


In [25]:
# Specify camera folders and output video path
camera_folders = [os.path.join(image_dir, f"C{i+1}") for i in range(7)]
output_video_path = os.path.join(dataset_path, "detection_output.mp4")

# Define video writer for saving output
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, 5, (640, 480))

# Process each frame in each camera folder
for cam_folder in camera_folders:
    image_files = sorted([f for f in os.listdir(cam_folder) if f.endswith('.png')])
    for image_file in image_files:
        image_path = os.path.join(cam_folder, image_file)
        image = cv2.imread(image_path)
        
        # Detect objects
        detections = detect_objects_in_image(image)
        
        # Draw bounding boxes
        image_with_boxes = draw_boxes_on_image(image, detections)
        
        # Resize if needed and write to video
        image_with_boxes_resized = cv2.resize(image_with_boxes, (640, 480))
        out.write(image_with_boxes_resized)

out.release()
print("Output video saved at:", output_video_path)


W0000 00:00:1730792576.769439    7105 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -2484 } dim { size: -2485 } dim { size: -2486 } dim { size: 1088 } } } inputs { dtype: DT_FLOAT shape { dim { size: -105 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -105 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 17 } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 2649 num_cores: 16 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 33554432 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { dim { size: -

KeyError: 'num_detections'

In [ ]:
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_video_path, fourcc, 5, (640, 480))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect objects
        detections = detect_objects_in_image(frame)
        
        # Draw bounding boxes
        frame_with_boxes = draw_boxes_on_image(frame, detections)
        
        # Resize if needed and write to video
        frame_with_boxes_resized = cv2.resize(frame_with_boxes, (640, 480))
        out.write(frame_with_boxes_resized)
    
    cap.release()
    out.release()
    print("Processed video saved at:", output_video_path)

# Example usage:
process_video("path/to/new_video.mp4", "path/to/output_new_video.mp4")
